# Task 2: Understand body language by gesture recognition with convolutional neural network

## 1. Do literature search on Convolution Neural Network. Learn how to build a convolutional layer in PyTorch.

## 2. Referring to the guide in Task 1, build your own network for gesture classification using convolutional layers. Please see the references 4 in the manual to learn how to build convolutional layers in PyTorch.

## 3. Analyse and comment on the performance of the model. Make a comparison between the fully connected based and convolutional based models and comment on it.

In [45]:
import cv2
import numpy as np
import os
import torch
import itertools
import torch.utils.data as utils_data
import torch.nn as nn

In [46]:
def processSkinImage(filePath, resize_HW=48):
    # step 1
    # read the image
    original = cv2.imread(filename=filePath)

    # step 2
    # resize the image to
    image_resized = cv2.resize(original, (resize_HW, resize_HW))

    # step 3
    # convert the image from rgb to YCbCr
    image_ycbcr = cv2.cvtColor(image_resized, cv2.COLOR_BGR2YCR_CB)

    # step 4
    # get the central color of the image
    # expected the hand to be in the central of the image
    Cb_center_color = image_ycbcr[int(resize_HW/2), int(resize_HW/2), 1]
    Cr_center_color = image_ycbcr[int(resize_HW/2), int(resize_HW/2), 2]
    # set the range
    Cb_Difference = 15
    Cr_Difference = 10

    # step 5
    # detect skin pixels
    Cb = image_ycbcr[:, :, 1]
    Cr = image_ycbcr[:, :, 2]
    index = np.where((Cb >= Cb_center_color-Cb_Difference) & (Cb <= Cb_center_color+Cb_Difference)
                     & (Cr >= Cr_center_color-Cr_Difference) & (Cr <= Cr_center_color+Cr_Difference))

    # Mark detected pixels and output
    image_output = np.zeros((resize_HW, resize_HW))
    image_output[index] = 255

    # show image
    # cv2.imshow("", image_output)
    # cv2.waitKey(0)
    return image_output

In [47]:
path = './dataset/dataset/images'
path_processed = './dataset_processed/images'

# -------------------images processing--------------
for mainDir, subDir, fileList in os.walk(path):
    for file in fileList:
        currentPath = os.path.join(mainDir, file)
        processedImage = processSkinImage(currentPath)

        new_mainDir = path_processed + mainDir.split(path)[-1]
        if not os.path.exists(new_mainDir):
            os.makedirs(new_mainDir)
        cv2.imwrite(os.path.join(new_mainDir, file), processedImage)

# -----------------label generation----------------
label_path = './dataset_processed/labels'
if not os.path.exists(label_path):
    os.makedirs(label_path)

files = os.listdir(path)
for i, file in enumerate(files):
    subclass_label_path = os.path.join(label_path, file+'.txt')
    with open(subclass_label_path, 'w') as f:
        f.write('#label\n')
    for _ in range(len(os.listdir(os.path.join(path_processed, file)))):
        with open(subclass_label_path, 'a') as f:
            f.write('{:d}\n'.format(i))
    f.close()

In [48]:
Image = []
path_images = './dataset_processed/images'
for mainDir, subDir, fileList in os.walk(path_images):
    for file in fileList:
        currentPath = os.path.join(mainDir, file)
        Image.append(cv2.imread(currentPath)[:, :, 0])

Image = torch.tensor(np.array(Image))
dataset_size, H, W = Image.shape
Image = torch.unsqueeze(Image, 1)
Image = np.array(Image)

Label = []
path_labels = './dataset_processed/labels'
for file in os.listdir(path_labels):
    Label.append(np.loadtxt(os.path.join(path_labels, file)))
Label = np.array(list(itertools.chain.from_iterable(Label)))
num_classes = int(np.max(Label))+1

(78, 1, 48, 48)


In [49]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        self.conv_layer1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=2) #(48-2)+1=47
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2) #(47-2)/2 +1= 23
        self.conv_layer2 = nn.Conv2d(in_channels=2, out_channels=4, kernel_size=2) #(23-2)+1=22
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2) #(22-2)/2+1=11
        
        self.fc1 = nn.Linear(11*11*4, 32)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(32, 4)
        
    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.max_pool1(out)
        out = self.conv_layer2(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        
        return out

In [50]:
model = CNNModel()
# if torch.cuda.is_available():
#     model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_func = nn.CrossEntropyLoss()

In [51]:
dataset = utils_data.TensorDataset(torch.Tensor(Image), torch.LongTensor(Label))
split_ratio = 0.8
train_size = int(split_ratio * dataset_size)
test_size = dataset_size - train_size
train_set, test_set = utils_data.random_split(dataset, [train_size, test_size])
train_loader = utils_data.DataLoader(dataset=train_set, batch_size=3, shuffle=True)
test_loader = utils_data.DataLoader(dataset=test_set, batch_size=3, shuffle=True)
print('Data is ready!')

Data is ready!


In [52]:
best_accuracy = 0
for epoch in range(500):
    running_loss = 0.0
    train_acc = 0.0
    for step, (batch_image, batch_label) in enumerate(train_loader):
        model.train()
        # if torch.cuda.is_available():
        #     batch_image, batch_label = batch_image.cuda(), batch_label.cuda()
        batch_output = model(batch_image)
        batch_loss = loss_func(batch_output, batch_label)

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()

        running_loss += batch_loss.item()

        # train accuracy
        _, train_predicted = torch.max(batch_output.data, 1)
        train_acc += (train_predicted == batch_label).sum().item()

    train_acc /= train_size
    running_loss /= (step+1)

    # ----------test----------
    model.eval()
    test_acc = 0.0
    for test_image, test_label in test_loader:
        test_output = model(test_image)
        _, predicted = torch.max(test_output.data, 1)
        test_acc += (predicted == test_label).sum().item()
    test_acc /= test_size

    print('epoch={:d}\ttrain loss={:.6f}\ttrain accuracy={:.3f}\ttest accuracy={:.3f}'.format(
        epoch, running_loss, train_acc, test_acc))

    if test_acc >= best_accuracy:
        torch.save(model.state_dict(), './trained_models/FCNN_model.pkl')
        best_accuracy = test_acc

epoch=0	train loss=2.822010	train accuracy=0.613	test accuracy=0.688
epoch=1	train loss=0.109050	train accuracy=0.968	test accuracy=0.688
epoch=2	train loss=0.090574	train accuracy=0.984	test accuracy=0.688
epoch=3	train loss=0.014111	train accuracy=1.000	test accuracy=0.750
epoch=4	train loss=0.002785	train accuracy=1.000	test accuracy=0.750
epoch=5	train loss=0.002447	train accuracy=1.000	test accuracy=0.750
epoch=6	train loss=0.001677	train accuracy=1.000	test accuracy=0.750
epoch=7	train loss=0.001047	train accuracy=1.000	test accuracy=0.750
epoch=8	train loss=0.000833	train accuracy=1.000	test accuracy=0.750
epoch=9	train loss=0.000660	train accuracy=1.000	test accuracy=0.750
epoch=10	train loss=0.000572	train accuracy=1.000	test accuracy=0.750
epoch=11	train loss=0.000489	train accuracy=1.000	test accuracy=0.750
epoch=12	train loss=0.000448	train accuracy=1.000	test accuracy=0.750
epoch=13	train loss=0.000401	train accuracy=1.000	test accuracy=0.750
epoch=14	train loss=0.000382	t